In [1]:
import pandas as pd
import numpy as np
import pandas_ta as pta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import data as dt
import functions as fn


In [2]:
BTCUSDT15m = dt.BTCUSDT15m
BTCUSDT15m.set_index("Open Time", inplace=True)

In [3]:
BTCUSDT15m['VWAP'] = fn.vwap(BTCUSDT15m.High, BTCUSDT15m.Low, BTCUSDT15m.Close, BTCUSDT15m.Volume)

In [4]:
train = BTCUSDT15m['2018-01-01 00:00:00': '2019-01-01 00:00:00']

In [5]:
t1 = BTCUSDT15m['2021-03-18 19:00:00': '2021-03-21 00:00:00']

In [6]:
fig = go.Figure(data=[go.Candlestick(x=t1.index,
                open=t1['Open'],
                high=t1['High'],
                low=t1['Low'],
                close=t1['Close']),
                go.Scatter(x=t1.index, y=t1.VWAP, line=dict(color='blue', width=1), name="VWAP")])
                
fig.update_yaxes(fixedrange=False)
fig.show()

# VWAP SIGNAL

In [7]:
vwap_signal = [0]*len(train)
back_candles = 9

In [8]:
for row in range(back_candles-1, len(train)):
    uptrend = 1
    downtrend = 1
    for i in range(row-back_candles, row+1):
        if train.Close[row]>=train.VWAP[row]:
            downtrend=0
        if train.Close[row]<=train.VWAP[row]:
            uptrend=0
    if uptrend==1 and downtrend==1:
        #print("!!!!! check trend loop !!!!")
        vwap_signal[row]=3
    elif uptrend==1:
        vwap_signal[row]=2
    elif downtrend==1:
        vwap_signal[row]=1

train['VWAP_signal'] = vwap_signal

C:\Users\rhg22\AppData\Local\Temp\ipykernel_21160\2107234758.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# PIVOTS SIGNAL

In [12]:
pivots = fn.pivots(train['Open'], train['High'], train['Low'], train['Close'], anchor=15, method='fibonacci')

In [9]:
pivs = fn.pivots(t1['Open'], t1['High'], t1['Low'], t1['Close'], anchor=15, method='fibonacci')

In [10]:
pivs

,p,s1,s2,s3,r1,r2,r3
Open Time,,,,,,,
2021-03-18 19:00:00,57695.906667,57163.566747,56834.686587,56302.346667,58228.246587,58557.126747,59089.466667
2021-03-18 19:15:00,57695.906667,57163.566747,56834.686587,56302.346667,58228.246587,58557.126747,59089.466667
2021-03-18 19:30:00,57695.906667,57163.566747,56834.686587,56302.346667,58228.246587,58557.126747,59089.466667
2021-03-18 19:45:00,57695.906667,57163.566747,56834.686587,56302.346667,58228.246587,58557.126747,59089.466667
2021-03-18 20:00:00,57695.906667,57163.566747,56834.686587,56302.346667,58228.246587,58557.126747,59089.466667
...,...,...,...,...,...,...,...
2021-03-20 23:00:00,58600.816667,57813.961607,57327.841727,56540.986667,59387.671727,59873.791607,60660.646667
2021-03-20 23:15:00,58600.816667,57813.961607,57327.841727,56540.986667,59387.671727,59873.791607,60660.646667
2021-03-20 23:30:00,58600.816667,57813.961607,57327.841727,56540.986667,59387.671727,59873.791607,60660.646667


In [11]:
fig = go.Figure(data=[go.Candlestick(x=t1.index,
                open=t1['Open'],
                high=t1['High'],
                low=t1['Low'],
                close=t1['Close']),
                go.Scatter(x=t1.index, y=t1.VWAP, line=dict(color='blue', width=1.5), name="VWAP"),
                go.Scatter(x=pivs.index, y=pivs.p, line=dict(color='red', width=1.5), name="P"),
                go.Scatter(x=pivs.index, y=pivs.s1, line=dict(color='orange', width=1), name="S1"),
                go.Scatter(x=pivs.index, y=pivs.s2, line=dict(color='orange', width=1), name="S2"),
                go.Scatter(x=pivs.index, y=pivs.r1, line=dict(color='orange', width=1), name="R1"),
                go.Scatter(x=pivs.index, y=pivs.r2, line=dict(color='orange', width=1), name="R2")])
                
fig.update_yaxes(fixedrange=False)
fig.show()